由于Cython是Python的超集，所以Python解释器不能直接导入和运行它，我们需要通过Cython编译管道(compilation pipeline)将Cython源码转换成可被Python调用的形式。

### Pipeline
- 1st stage：Cython源码$\Longrightarrow$平台独立的优化C或C++代码。
- 2nd stage：生成的C或C++源码$\Longrightarrow$可被Python调用的库(extension module)。

#### 标准方法：使用distutils和cythonize
需要编写一个Python脚本并运行它
- 1st stage：Cython的cythonize方法。返回a list of distutils Extension objects
- 2nd stage：distutils中的setup方法
##### 执行该脚本
!python setup.py build_ext --inplace --compiler=msvc

In [5]:
%%file fib.pyx
def cyfib(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a+b, a
    return a

Overwriting fib.pyx


In [8]:
%%file setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize
setup(ext_modules=cythonize('fib.pyx'))

Overwriting setup.py


In [10]:
# 执行setup.py脚本，生成可被调用的fib.pyd
!python setup.py build_ext --inplace --compiler=msvc

running build_ext
building 'fib' extension
creating build
creating build\temp.win-amd64-2.7
creating build\temp.win-amd64-2.7\Release
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\cl.exe /c /nologo /Ox /MD /W3 /GS- /DNDEBUG "-ID:\Program Files\Anaconda\include" "-ID:\Program Files\Anaconda\PC" /Tcfib.c /Fobuild\temp.win-amd64-2.7\Release\fib.obj
fib.c
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\link.exe /DLL /nologo /INCREMENTAL:NO "/LIBPATH:D:\Program Files\Anaconda\libs" "/LIBPATH:D:\Program Files\Anaconda\PCbuild\amd64" /EXPORT:initfib build\temp.win-amd64-2.7\Release\fib.obj "/OUT:D:\Program Files\Anaconda\QieLearning\Cython\Chapter2 Compiling and Running Cython Code\fib.pyd" /IMPLIB:build\temp.win-amd64-2.7\Release\fib.lib /MANIFESTFILE:build\temp.win-amd64-2.7\Release\fib.pyd.manifest
fib.obj : warning LNK4197: export 'initfib' specified multiple times; using first specific

In [11]:
import fib
fib.cyfib(10)

144

In [12]:
# 将生成的文件打包写入zip文件
import zipfile
f = zipfile.ZipFile('chapter2_fib.zip','w',zipfile.ZIP_DEFLATED)
names = 'fib.pyd fib.pyx fib.c setup.py'.split()
for name in names:
    f.write(name)
f.close()

#### 使用ipython的%%cython魔法语句实现交互式Cython
使用distutils的一大缺陷是需要独立的编译步骤，无法实现交互。而使用ipython的魔法命令可以很方便的在交互式环境下使用Cython。
- %load_ext Cython : 加载Cython的魔法命令
- %%cython : 使我们可以在IPython解释器中直接编写Cython代码块。IPython将我们定义的Cython代码复制进Cython源码，然后将其编译成一个extension module。如果编译时成功的，IPython将导入该module中的everything到interactive namespace。我么可以传入一些可选参数给该魔术命令：
    - -n, --name: 指定生成的.pyx文件的名字
    - --cplus: 生成C++源码
    - -a, --annotate: 输出一个annotated source file
    - -f, --force: 强制cython重新生成C或C++源码
    - -I, --include: adds directories to search for file inclusion and *cimports*
    - -c, --compile-args: allow inclusion of extra C compiler arguments
    - --link-args: allow inclusion of extra link arguments
    - -L: adds extra library search directories
    - -l: adds extra library names to link against
- %%cython_inline: 编译并运行嵌入在当前Python命名空间中的Cython代码
- %%cython_pyximport: 建立在pyximport package之上

In [14]:
%load_ext Cython

In [16]:
%%cython
def fib(int n):
    cdef int i
    cdef double a = 0.0, b = 1.0
    for i in range(n):
        a, b = a+b, a
    return a

In [19]:
%timeit fib(90)

The slowest run took 7.94 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 138 ns per loop


#### 通过pyximport库实时动态的进行编译
- pyximport 改进了import使其可以识别.pyx扩展模块，自动将它们发送到compilation pipeline，然后import这些扩展模块在python中使用。pyximport使我们可以无需编写额外的脚本，并将.pyx模块当做普通的python模块使用。
- 如果一个Cython源文件被修改，则pyimport将会自动探测到修改，当其在一个新的python interpreter session中被导入时重新编译源文件。
- 因为通过pyximport导入的Cython模块依赖于cython compiler以及一个C compiler，其不适合用在依赖关系不在我们掌控之下的生产环境中。

In [24]:
import pyximport
pyximport.install()
import fib
fib.cyfib(10)

144